In [1]:
import cv2
from glob import glob
import numpy as np
import pandas as pd
from time import time
import ast
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import argparse
import imutils
import random
import math
import os
import dlib
from scipy.interpolate import interp2d

from phase1.Misc.utils import *
from phase1.dTriangulation import *
from phase1.ThinPlateSpline import *

from phase2.api_ import PRN_
from phase2.api import PRN
from phase2.prnet import *

from utils.render import render_texture
from utils.MovingAverage import *

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 



Num GPUs Available:  1
################################################ RUNNING IN GPU 


In [41]:
def videoRead(path = './Data/Ball_travel_10fps.mp4'):
    imgs = []
    cap = cv2.VideoCapture(path)
    frame_width = int(cap.get(3)) 
    frame_height = int(cap.get(4))    
    result = cv2.VideoWriter('video1.mp4',  
                        cv2.VideoWriter_fourcc(*'DIVX'), 
                        10, (frame_width, frame_height)) 

    while(True):
        ret, frame = cap.read()
        if ret:
#             frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = imutils.rotate(frame, 90)
            
#             frame  = cv2.resize(frame, (800,640))
            result.write(np.uint8(frame))
        else:
            break
    cap.release()
    result.release()
videoRead('/home/gokul/video.mp4')

In [35]:
# plt.imshow(imgs[0])
imgs[0].shape[0]

352

In [36]:
out = cv2.VideoWriter('Data1.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 15, (imgs[0].shape[0],imgs[0].shape[1]))
for i in range(len(imgs)):
    out.write(imgs[i])
out.release()

# Do for an Image Pair

In [3]:
path_to_shape_predictor = './phase1/Misc/shape_predictor_68_face_landmarks.dat'
path_to_image = "../Data/"
path_to_image1 = path_to_image+ "Face1.jpg"
path_to_image2 =  path_to_image + "Face2.jpg"

Face1, Face2 = readImages(path_to_image1, path_to_image2)
grayFace1 = cv2.cvtColor(Face1, cv2.COLOR_BGR2GRAY)
grayFace2 = cv2.cvtColor(Face2, cv2.COLOR_BGR2GRAY)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(path_to_shape_predictor)

    
# get bounding box of face
N_pyrLayers = 1
box1 = detector(grayFace1,N_pyrLayers)[0]
# box2 = detector(grayFace2,N_pyrLayers)[0]
try:
    box2 = detector(grayFace2,N_pyrLayers)[0]
except IndexError as e:
    print(e)
    sys.exit(1) 

############################# For any two given Faces, do swapping #############################
method = 'DelTri'

## detect the facial markers
if method == 'TPS':
    print('TPS')
    WarpedFace, Face1_print, Face2_print = FaceSwap1_TPS(Face1,Face2, box1,box2, predictor)

elif method == 'DelTri':
    print('DelTri')
    WarpedFace, Face1_print, Face2_print = FaceSwap1_delTri(Face1,Face2, box1,box2, predictor)

# elif method = 'PrNet':
    

_, plts = plt.subplots(1,3,figsize=(15,5))
plts[0].imshow(Face1_print)
plts[1].imshow(Face2_print)
plts[2].imshow(WarpedFace)

DelTri


TypeError: FaceSwap1_delTri() missing 5 required positional arguments: 'use_filter', 'first_time', 'old_markers', 'moving_average_position', and 'moving_average_del'

### Face on video destination

In [ ]:
# def smoothenFrames(current_frame, past_frame):
#     new_frame = (0.4 * past_frame + 0.6 * current_frame).astype(int)
#     del_frame = new_frame - past_frame
#     del_frame[del_frame > 100] = 100
#     # del_frame[del_frame < 100] = -100
#     new_frame = past_frame + del_frame
    
#     return np.uint8(new_frame), del_frame

In [ ]:
# def adjustGamma(image, gamma=1.0):
# 	# build a lookup table mapping the pixel values [0, 255] to
# 	# their adjusted gamma values
# 	invGamma = 1.0 / gamma
# 	table = np.array([((i / 255.0) ** invGamma) * 255
# 		for i in np.arange(0, 256)]).astype("uint8")
# 	# apply gamma correction using the lookup table
# 	return cv2.LUT(image, table)

In [2]:
def predictBoxes(past_box, del_box):
    pass


In [2]:
def getBBCorners(box):
    (x,y,h,w) = box
    corners = []
    corners.append([x, y])
    corners.append([x + h, y])
    corners.append([x + h, y + w])
    corners.append([x, y + w])
    return corners


In [3]:
def getDelBox(current_box, past_box = None):
    if past_box is None:
        return None
    bb_current = getcv2Box(current_box)
    bb_past = getcv2Box(past_box)
    
    

In [4]:
path_to_shape_predictor = './phase1/Misc/shape_predictor_68_face_landmarks.dat'
DataPath = "../Data/TestSet/"
RefImageFileName = 'Rambo.jpg'

RefImageFilePath = DataPath + RefImageFileName
VideoFilePath = DataPath + "Test1.mp4"

SavePath = "../Results/PRNet/"            

SaveFileName = SavePath + "output_rambo_unfiltered.mp4"
method = 'PRNet'
use_filter = True

if(not (os.path.isdir(SavePath))):
    print(SavePath, "  was not present, creating the folder...")
    os.makedirs(SavePath) 
    

##### Setting up reference image ######
print('Setting up reference image.......')
FaceRef = cv2.imread(RefImageFilePath) ## color image
if FaceRef is None:
    mode = 2
else:
    mode = 1
print("we are in mode ", mode, " using ", method)
    # ref_h,ref_w = FaceRef.shape[:2]
# grayFaceRef = cv2.cvtColor(FaceRef, cv2.COLOR_BGR2GRAY)

##### Setting up video ######
cap = cv2.VideoCapture(VideoFilePath)
frame_width = int(cap.get(3)) 
frame_height = int(cap.get(4))
result = cv2.VideoWriter(SaveFileName,  
                        cv2.VideoWriter_fourcc(*'DIVX'), 
                        10, (frame_width, frame_height)) 

##############################################################################
########################### mode 1 ###########################################
##############################################################################
if mode == 1:
    print("Doing Mode 1 with Reference Image")
    
    ##################################################
    ########### Setup pretrained models ##############
    ##################################################
    if method != 'PRNet':
        detector = dlib.get_frontal_face_detector()
        predictor = dlib.shape_predictor(path_to_shape_predictor)
        
        grayFaceRef = cv2.cvtColor(FaceRef, cv2.COLOR_BGR2GRAY) ## gray image
        N_pyrLayers = 1
        boxRef = detector(grayFaceRef,N_pyrLayers) ## reference bounding box
        if len(boxRef)!=1:
            print('Cannot find face in reference Image... exiting... :( ', len(boxRef))
            exit()
        else:
            boxRef = boxRef[0]
        ##ref image features
        FaceMarks_ref, BoundingBox_ref, shifted_FaceMarks_ref = FaceDetector(FaceRef, boxRef, predictor)
        (x,y,w,h) = BoundingBox_ref
        FaceRef_crop  = FaceRef[y:y+h,x:x+w]
            
    elif method == 'PRNet':
        prn = PRN(is_dlib = True)    
        
        
    # ##### Setting up video ######
    # cap = cv2.VideoCapture(VideoFilePath)
    # frame_width = int(cap.get(3)) 
    # frame_height = int(cap.get(4))
    # result = cv2.VideoWriter(SaveFileName,  
    #                         cv2.VideoWriter_fourcc(*'DIVX'), 
    #                         10, (frame_width, frame_height)) 

    facemarksFrames = []
    #for filtering
    old_box = None
    del_boxFrame = 0
    first_time = True

    moving_average_position = MovingAverage(window_size = 4, weight = [1, 1, 1, 1])
    moving_average_del = MovingAverage(window_size = 4, weight = [1, 1, 1, 1])
    FaceMarkersFrame = 0
    ## run the loop
    while(True):    
        ret, frame = cap.read()
        if not ret:
            print("Stream ended..")
            break
        frame = adjustGamma(frame, 1.5)
        frame[frame > 255] = 255

        # frame = cv2.resize(frame, (ref_w,ref_h))
        
        if method == 'PRNet':
            
            pos = prn.process(frame)
            ref_pos = prn.process(FaceRef)
            
            if pos is None:
                if prev_pos is not None:
                    pos = prev_pos
                else:
                    print("## No face Found ##")
                    WarpedFace = frame
    #                 text_location = (frame_width-int(frame_width*0.2), frame_height- frame_height*0.1)
    #                 WarpedFace = cv2.putText(WarpedFace, 'Cannot Find Face',text_location, cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0) ,3, cv2.LINE_AA, False)
            if pos is not None:
    #             if len(pos)>1:
    #                 print('Wrong Mode chosen')
    #                 exit()
    #             elif len(pos)==1:
                WarpedFace = FaceSwap_DL(prn, pos, ref_pos, frame, FaceRef)

        #####################################################################
        ######################## Traditional methods ########################
        else:
            grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            boxFrame = detector(grayFrame, N_pyrLayers)
            
            #Check if face is detected
            if len(boxFrame)>1:
                margin = 50
                boxRef,boxFrame  = boxFrame[0], boxFrame[1]
                FaceFrame = FaceFrame[rects[1].top()-margin:rects[1].bottom()+margin,rects[1].left()-margin:rects[1].right()+margin,:]
                FaceRef = FaceRef[rects[0].top()-margin:rects[0].bottom()+margin,rects[0].left()-margin:rects[0].right()+margin,:]
                
            elif(len(boxFrame) < 1):
                if not first_time:
                    print("no box detected, using old box")
                    # boxFrame.append(old_boxFrame[0] + del_boxFrame)
                    boxFrame = old_boxFrame
                else:
                    print("No face found")
                    WarpedFace = frame 
            ########################################

            #face swaping
            if (len(boxFrame) > 0): 
                # if not first_time:                
                #     del_boxFrame = boxFrame[0] - old_boxFrame[0]
                #     del_flag = True
                # del_box = getDelBox(boxFrame, old_boxFrame)
                old_boxFrame = boxFrame       
                boxFrame = boxFrame[0]            

                if method == 'TPS':

                    WarpedFace, FaceMarkersFrame = FaceSwap1_TPS(FaceRef_crop, shifted_FaceMarks_ref, frame, boxFrame, predictor, use_filter, first_time, FaceMarkersFrame, moving_average_position, moving_average_del)
                    #frame smoothening
                    if use_filter:
                        if not first_time:
                            # WarpedFace, del_frame = smoothenFrames(WarpedFace, past_frame)
                            past_frame = WarpedFace
                        else:
                            past_frame = WarpedFace

                    first_time = False

                elif method == 'DelTri':

                    WarpedFace, FaceMarkersFrame = FaceSwap1_delTri(FaceRef, FaceMarks_ref, frame, boxFrame, predictor, use_filter, first_time, FaceMarkersFrame, moving_average_position, moving_average_del)
                    #frame smoothening
                    if use_filter:
                        if not first_time:
                            WarpedFace, del_frame = smoothenFrames(WarpedFace, past_frame)
                            past_frame = WarpedFace
                        else:
                            past_frame = WarpedFace

                    first_time = False
                
            #######################
        cv2.imshow('frame', WarpedFace)
        result.write(WarpedFace)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
# MODE2_
else:
    ##################################################
    ########### Setup pretrained models ##############
    ##################################################
    print("Doing Mode 2 Swap within frame")
    print(method)
    if method == 'PRNet':
        prn = PRN_(is_dlib = True)       
    else:
        detector = dlib.get_frontal_face_detector()
        predictor = dlib.shape_predictor(path_to_shape_predictor)

    ##################################################
    ################# Setup Video ####################
    ##################################################
    # print("setting up the video")
    # cap = cv2.VideoCapture(VideoFilePath)
    # frame_width = int(cap.get(3)) 
    # frame_height = int(cap.get(4))
    # result = cv2.VideoWriter(SaveFileName,  
    #                         cv2.VideoWriter_fourcc(*'DIVX'), 
    #                         10, (frame_width, frame_height))     

    facemarksFrames = []
    ##################################################
    ################ Run loop ########################
    ##################################################
    outputs = []
    print("Begin swapping")
    while(True):    
        ret, frame = cap.read()

        if not ret:
            print("Stream ended..")
            break

        if method == 'PRNet':
            
            poses = prn.process(frame)
            if poses is None:
                poses = prev_poses
            print("number of Faces found...", len(poses))
            if len(poses)<2:
                poses = prev_poses
            if len(poses) == 2:
                prev_poses = poses
                pose1 ,pose2 = poses[0],poses[1]
                WarpedFace_tmp = FaceSwap_DL(prn, pose1, pose2, frame, frame)
                WarpedFace = FaceSwap_DL(prn, pose2, pose1, WarpedFace_tmp, frame)
            else:
                WarpedFace = frame
                text_location = (frame_width-400, frame_height-42)
                WarpedFace = cv2.putText(WarpedFace, 'Cannot Find Face',text_location, cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0) ,3, cv2.LINE_AA, False)        

        else:
            boxes = Top2Boxes(detector(gray(frame), 1))
            print("number of found...", len(boxes))
 
            if boxes is None or len(boxes) < 2:
                if not first_time:
                    boxes = old_box
            if boxes is not None and len(boxes) ==2: #review
                old_box = boxes
                # prev_boxes = boxes
                box1, box2 = boxes[0], boxes[1]
                if method == 'DelTri':
                    WarpedFace = FaceSwap2_delTri(frame,boxes[0], boxes[1], predictor)
                    first_time = False
                elif method == 'TPS':
                    WarpedFace = FaceSwap2_TPS(frame,boxes[0], boxes[1], predictor)
                    first_time = False
            else:
                WarpedFace = frame
                text_location = (frame_width-400, frame_height-42)
                WarpedFace = cv2.putText(WarpedFace, 'Cannot Find Face',text_location, cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0) ,3, cv2.LINE_AA, False)        

        cv2.imshow(str(method), WarpedFace)
        result.write(np.uint8(WarpedFace))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
result.release() 
cv2.destroyAllWindows()

Setting up reference image.......
we are in mode  1  using  PRNet
Doing Mode 1 with Reference Image




Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./phase2/Data/net-data/256_256_resfcn256_weight
Stream ended..
